## **Project 2: Hyperparameter Optimization**

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Convolution2D, MaxPooling2D, Flatten, Conv2D, Dropout, BatchNormalization
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing import image

In [2]:
trainDIR = '/workspace/seg_train/seg_train/'
testDIR = '/workspace/seg_test/seg_test/'
predDIR = '/workspace/seg_pred/seg_pred'

In [3]:
def get_images(directory):
    Images=[]
    Labels=[]
    
    for labels in os.listdir(directory):
        if labels == 'glacier':
            label = 0
        elif labels == 'sea':
            label = 1
        elif labels == 'buildings':
            label = 2
        elif labels == 'forest':
            label = 3
        elif labels == 'mountain':
            label = 4
        elif labels == 'street':
            label = 5
        
        for file in os.listdir(directory+labels):
            im =  image.load_img(directory+labels+r'/'+file, target_size=(150,150))
            img = image.img_to_array(im)
            Images.append(img)
            Labels.append(label)
    
    Images, Labels = sklearn.utils.shuffle(Images, Labels, random_state=4)
    return Images, Labels

In [4]:
Images, Labels = get_images(trainDIR)
Images = np.array(Images)
Labels = np.array(Labels)

In [5]:
train = Images[:3000]
train_labels=Labels[:3000]

In [6]:
test, test_labels = get_images(testDIR)
test = np.array(test)
test_labels = np.array(test_labels)

In [7]:
test.shape

(3000, 150, 150, 3)

In [8]:
def create_model(lr=0.01, nl1 = 1, nl2 = 1,
                 nn1 = 32, nn2 = 500):
    
    opt = keras.optimizers.Adam(lr=lr)
                                                     
    model = Sequential()
    
    for i in range(nl1):
        if i == 0:
            model.add(Convolution2D(nn1,(3,3), input_shape=(150,150,3), activation='relu'))
            model.add(MaxPooling2D((2,2)))
        else:
            model.add(Convolution2D(nn1,(3,3), activation='relu'))
            model.add(MaxPooling2D((2,2)))
        model.add(Flatten())
        nn1/=2
        
    for i in range(nl2):
        model.add(Dense(nn2, activation='relu'))
        nn2/=2
    
    model.add(Dropout(0))   
    model.add(Dense(6, activation='softmax'))
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model


In [9]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# model class to use in the scikit random search CV 
model = KerasClassifier(build_fn=create_model, epochs=5, batch_size=32, validation_split=0.25)

In [10]:
# learning rate parameters
lr=[1e-2, 1e-3]

# of neurons in each layer
nn1 = [64,32]
nn2 = [1000,500]

# of convolutional and maxpooling layers
nl1 = [1,2]

# of dense layers (exlcusing output layer)
nl2 = [1,2]

# dictionary summary
param_grid = dict(nn1= nn1, nn2 = nn2, lr=lr, nl1=nl1, nl2=nl2)

In [11]:
from sklearn.model_selection import RandomizedSearchCV, KFold

grid = RandomizedSearchCV(estimator=model, cv=KFold(3), param_distributions=param_grid, n_iter=3, n_jobs=1)

In [12]:
grid_result = grid.fit(train, train_labels)

Epoch 1/5
47/47 [==============================] - 1s 21ms/step - loss: 4864.3843 - accuracy: 0.2047 - val_loss: 1.8901 - val_accuracy: 0.1620
Epoch 2/5
47/47 [==============================] - 1s 15ms/step - loss: 1.7662 - accuracy: 0.2040 - val_loss: 1.8529 - val_accuracy: 0.1660
Epoch 3/5
47/47 [==============================] - 1s 15ms/step - loss: 1.7026 - accuracy: 0.2387 - val_loss: 1.8555 - val_accuracy: 0.1760
Epoch 4/5
47/47 [==============================] - 1s 15ms/step - loss: 1.6783 - accuracy: 0.2573 - val_loss: 1.8719 - val_accuracy: 0.1620
Epoch 5/5
32/32 [==============================] - 0s 6ms/step - loss: 1.9675 - accuracy: 0.1680
Epoch 1/5
47/47 [==============================] - 1s 16ms/step - loss: 9286.1875 - accuracy: 0.1793 - val_loss: 1.9170 - val_accuracy: 0.1840
Epoch 2/5
47/47 [==============================] - 1s 15ms/step - loss: 1.8101 - accuracy: 0.1800 - val_loss: 1.8758 - val_accuracy: 0.1880
Epoch 3/5
47/47 [==============================] - 1s 15m

In [13]:
print(grid_result.best_params_)

{'nn2': 1000, 'nn1': 32, 'nl2': 2, 'nl1': 1, 'lr': 0.001}


In [14]:
best_model = grid_result.best_estimator_.model

In [15]:
best_model.evaluate(test,test_labels)

94/94 [==============================] - 1s 6ms/step - loss: 8.5759 - accuracy: 0.5533


[8.575867652893066, 0.5533333420753479]